# Imports  

In [ ]:
from gliderad2cp import process_currents, process_shear, process_bias, tools, download_example_data
import cmocean.cm as cmo
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np

### Settings

There are several options that can - and should - be customised including:
- QC correlation, amplitude and velocity thresholds
- Resolution of gridded output
- Offsets to correct for transducer misalignment
- Shear bias regression depths

Others that can - and maybe shouldn't - be customised:
- Distance weighting of shear bias corrections
- Velocity dependence of shear bias corrections

These are all set in the `options` dict. Running the `get_options` functions returns a default set of options, which are a great place to start.

In the example below, we grid per 3-profiles as we only require approx. half hour resolution. We restrict the shear-bias regression to depths deeper than 10m to eliminate potential effects of surface circulation causing velocity variance independent of instrument shear bias.

In [ ]:
options = tools.get_options(xaxis=3, yaxis=None, shear_bias_regression_depth_slice=(10,1000))

# Load data

`gliderad2cp` requires a netCDF file from a Nortek AD2CP which can be produced using the Nortek MIDAS software and a timeseries of glider data. This timeseries can be read from a netCDF, csv, or parquet file, or passed in as an xarray DataSet or pandas Dataframe. The essential variables are:

- time
- pressure
- temperature
- salinity
- latitude
- longitude
- profile_number

There are several example datasets available from the function `load_sample_dataset`. We use one of the [VOTO example datasets from the Baltic](https://observations.voiceoftheocean.org/SEA055/M82) in this notebook

In [ ]:
data_file = download_example_data.load_sample_dataset(dataset_name="sea055_M82.nc")
adcp_file = download_example_data.load_sample_dataset(dataset_name="sea055_M82.ad2cp.00000.nc")

# Step 1: calculate velocity shear

This is handled by the wrapper function `process_shear.process()`. The individual steps of processing are detailed in the [documentation](https://www.flow-lab.org/gliderad2cp/).

The output of this function is a gridded xarray dataset including data from the AD2CP like ensemble correlation and return amplitude, as well as calculated glider relative velocities and profiles of eastward, northward, and vertical velocities SH_E, Sh_N and Sh_U.

In [ ]:
ds_adcp = process_shear.process(adcp_file, data_file, options)

### Plots

The output of `process_shear.process()` can be plotted to visually examine per-beam, per-bin values. This can be useful for fine tuning QC settings in `options`.

Here we observe velocity measured along the X-direction of the ADCP, as a function of both time and distance from the glider; this would be equivalent to the opposite of the glider's speed through water if it had no angle of attack.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
mappable = ax.pcolormesh(ds_adcp.time, ds_adcp.gridded_bin, ds_adcp.X.T, cmap='PiYG', vmin=-1, vmax=1)
fig.colorbar(ax=ax,mappable=mappable, label='X velocity (along glider, m.s$^{-1}$)')
ax.set(ylabel='Bin number', title='Return amplitude Beam 2', xlim=(np.datetime64("2024-10-11T10"), np.datetime64("2024-10-11T12")))
ax.invert_yaxis()


# Step 2: shear to velocity

After calculating velocity shear, this can be integrated and referenced to estimate earth-relative velocity profiles.

The function `process_currents.process` handles this step, returning DAC-referenced velocity profiles.

### Prerequisite: Get pre- and post-dive GPS locations from glider data

Referencing is done against dive-averaged currents, which provided an estimate of the barotropic component of the velocity profile. To calculate dive-averaged currents, we opt to use the instrument as a doppler-velocity logger. If we input both the pre- and post-dive GPS locations of the glider, we can calculate both the displacement through water (from DVL) and the displacement over earth (from GPS); the difference divided by dive-duration is the estimate of dive-averaged currents.

As all gliders have different GPS formats, we leave it to you to provide coordinates and timestamps for dives and surfacings. The code is able to cope with consecutive no-surface dives as well as loiter dives by weighting the baroclinic profiles before referencing to dive-averaged currents.

See the documentation for more examples of this calculation and make use of the verification plots. Alternatively, one can manually reference the output of the `process_currents._grid_shear` and `process_currents._grid_velocity` functions. This can be useful if one wishes to use a level of no motion, bottom tracking (which will be implemented in a later release) or surface drift.

In [ ]:
data = xr.open_dataset(data_file)
gps_predive = []
gps_postdive = []

dives = np.round(np.unique(data.dive_num))

_idx = np.arange(len(data.dead_reckoning.values))
dr  = np.sign(np.gradient(data.dead_reckoning.values))

for dn in dives:
    _gd = data.dive_num.values == dn
    if all(np.unique(dr[_gd]) == 0):
        continue

    _post = -dr.copy()
    _post[_post != 1] = np.nan
    _post[~_gd] = np.nan

    _pre = dr.copy()
    _pre[_pre != 1] = np.nan
    _pre[~_gd] = np.nan

    if any(np.isfinite(_post)):
        # The last -1 value is when deadreckoning is set to 0, ie. GPS fix. This is post-dive.
        last  = int(np.nanmax(_idx * _post))
        gps_postdive.append(np.array([data.time[last].values, data.longitude[last].values, data.latitude[last].values]))

    if any(np.isfinite(_pre)):
        # The first +1 value is when deadreckoning is set to 1, the index before that is the last GPS fix. This is pre-dive.
        first = int(np.nanmin(_idx * _pre))-1 # Note the -1 here.
        gps_predive.append(np.array([data.time[first].values, data.longitude[first].values, data.latitude[first].values]))

gps_predive = np.vstack(gps_predive)
gps_postdive = np.vstack(gps_postdive)

We expect `gps_postdive` and `gps_predive` to show as vertical blue and red lines respectively at the beginning and end of a glider surfacing manouvre. In a mission with multiple no-surface dives, as shown in the example below, the dives where the gliders does not surface to fix GPS, do not get assigned `gps_postdive` and `gps_predive`.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(data.time, data.depth, color='k')
for predive in gps_predive:
    ax.axvline(predive[0], color='r')
ax.axvline(predive[0], color='r', label='GPS predive')

for postdive in gps_postdive:
    ax.axvline(postdive[0], color='b')
ax.axvline(postdive[0], color='b', label='GPS postdive')
ax.invert_yaxis()
ax.set(xlim=(np.datetime64("2024-10-11T00:30"), np.datetime64("2024-10-11T03:30")), ylabel='Depth (m)')
ax.legend();

In [ ]:
currents, DAC = process_currents.process(ds_adcp, gps_predive, gps_postdive, options)

### Plot DAC referenced currents

We can plot the DAC-referenced eastward and northward velocities.

In [ ]:

plt.figure(figsize=(20,6))

plt.subplot(121)
plt.pcolormesh(currents.velocity_E_DAC_reference, cmap=cmo.balance)
plt.gca().invert_yaxis()
plt.colorbar()
plt.clim([-0.6, 0.6])

plt.subplot(122)
plt.pcolormesh(currents.velocity_N_DAC_reference, cmap=cmo.balance)
plt.gca().invert_yaxis()
plt.colorbar()
plt.clim([-0.6, 0.6])

# Step 3: Estimate and correct shear bias

This optional processing step attempts to correct for along-beam shear bias. Shear-bias is described in Todd et al. 2017 (JAOTECH, https://doi.org/10.1175/JTECH-D-16-0156.1), section 3.2.b.

Shear-bias is the result of very small shear present in beams during individual pings. It is not yet known what causes it although reports from Nortek and others suggest an instrument dependence. It is also reduced with stricter signal-to-noise ratio thresholds. As shear is exagerated throughout the water-column, the error in velocity grows with the depth of the profile. It is generally visible as an erroneous supplement velocity component aligned with the glider's direction of travel.

As it grows with profile-depth, it is almost inconsequential in this example but we do it to serve as an example.

In [ ]:
currents = process_bias.process(currents,options)

# Compare outputs

The following three plots contrast the eastward velocities estimates at three crutical points of processing:
1. From integration of shear
2. After referencing integrated shear profiles to DAC
3. After applying the shear bias correction to DAC referenced velocities 

In [ ]:
currents

In [ ]:
variables = ["velocity_E_no_reference", "velocity_E_DAC_reference", "velocity_E_DAC_reference_sb_corrected"]
titles = ["No referencing", "DAC referencing", "DAC referencing, bias correction"]
fig, axs = plt.subplots(3, 1, figsize=(10,14))
for i in range(3):
    ax = axs[i]
    mappable = ax.pcolormesh(currents.time[:-1], currents.depth, currents[variables[i]][:, :-1], cmap=cmo.balance, vmin=-0.6, vmax=0.6)
    ax.invert_yaxis()
    fig.colorbar(ax=ax,mappable=mappable, label='Eastward velocity (m/s)')
    ax.set(ylabel='Depth (m)', title=titles[i])